In [ ]:
# -*- coding: utf-8 -*-
"""Solve.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1Y2T_sa0H260BcTQsSzTwNXxZQ6uOIUH-

## Acceso a Drive
"""

from google.colab import drive
drive.mount('/content/drive')

import re
import string
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install unidecode

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from unidecode import unidecode

In [ ]:
"""## Funciones"""

def clean_text(text):
    # Expresión regular para encontrar todas las cadenas que empiezan con "https://t.co/" y tienen uno o más caracteres que no son espacios en blanco.
    pattern = r'https://t.co/[^\s]+|@[a-zA-Z0-9_]{1,15}|[\U0001F600-\U0001F64F\U0001F300-\U0001F5FF\U0001F680-\U0001F6FF\U0001F1E0-\U0001F1FF]+|[^\w\s]'
    
    # Reemplaza todas las cadenas que coinciden con la expresión regular por una cadena vacía.
    clean_text = re.sub(pattern, '', text)
    
    # Convertir todas las letras a minúsculas
    clean_text = clean_text.lower()

    return clean_text

"""## Leyendo datos"""

text_df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/NLPFundamentals/Contest/sources/csvs/train_data.csv", names=['image', 'text'],header=None) 
labels_df = pd.read_csv("/content/drive/MyDrive/ColabNotebooks/NLPFundamentals/Contest/sources/csvs/train_labels_subtask_1.csv", names=['labels'],header=None)

d = {'image':text_df['image'].values, 'text':text_df['text'].values, 'labels':labels_df['labels'].values}
df = pd.DataFrame(data = d)

pd.set_option('display.max_colwidth',None)



"""## Limpieza de datos"""

df_Preprocessing = df.copy()

df_Preprocessing.drop('image', axis=1, inplace=True)

# Aplica la función a la columna "text" del DataFrame "df".
df_Preprocessing['text'] = df_Preprocessing['text'].apply(clean_text)


"""## Análisis exploratorio de datos

"""#### Undersampling"""

from imblearn.under_sampling import RandomUnderSampler
undersample = RandomUnderSampler(random_state=42)

X = df_Preprocessing.drop('labels',axis=1)
Y = df_Preprocessing.labels

X_over , Y_over = undersample.fit_resample(X,Y)

Y_over = pd.DataFrame(data=Y_over)

dNew = {'text':X_over['text'].values, 'labels':Y_over['labels'].values}

# NO ME GUSTA QUE LOS DATOS ESTÁN ORDENADOS
df_Balanced = pd.DataFrame(data = dNew)

# mezclar los registros del DataFrame
df_Balanced = df_Balanced.sample(frac=1, random_state=42) # 42 es una semilla aleatoria para replicar el resultado


"""## Divide el dataset"""

from sklearn.model_selection import train_test_split

train, test = train_test_split(df_Balanced, test_size=0.1, stratify=df_Balanced["labels"], random_state=42)
len(train), len(test)

df_Balanced_train = train["text"]
target_train = train["labels"]

df_Balanced_test = test["text"]
target_test = test["labels"]

"""## Feature engineering"""

example_sentence = df_Balanced.iloc[431]["text"]

from nltk.tokenize.toktok import ToktokTokenizer
tk_tokenizer = ToktokTokenizer()

tokens = tk_tokenizer.tokenize(example_sentence)

nltk.download('stopwords')

sp_stopwords = stopwords.words("spanish")
sp_punctuation = string.punctuation + '¿¡'

not_wanted = set((unidecode(word) for word in sp_stopwords)) | set(sp_punctuation)

tk_tokenizer = ToktokTokenizer()


def tokenize(sentence):
    clean = []
    clean_sentence = unidecode(sentence)
    for token_ in tk_tokenizer.tokenize(clean_sentence):
        token = token_.lower()
        if token in not_wanted:
            continue
        clean.append(token)
    return clean

tokens = tokenize(example_sentence)
tokens = tokenize(example_sentence)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizador_real = CountVectorizer(binary=True, analyzer=tokenize)

vectorizador_real.fit(df_Balanced_train)

train_x = vectorizador_real.transform(df_Balanced_train)
test_x = vectorizador_real.transform(df_Balanced_test)

### Implementacion de distintos modelos con Validacion cruzada

In [ ]:
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

#### Naive Bayes

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
# Crear el clasificador Naive Bayes
naive_bayes = MultinomialNB()

# Entrenar el clasificador con todos los datos de entrenamiento
naive_bayes.fit(train_x, target_train)

# Predecir las etiquetas de los datos de entrenamiento
train_predictions = naive_bayes.predict(train_x)
train_accuracy = accuracy_score(target_train, train_predictions)
print(f"Train accuracy:   {train_accuracy:0.2%}\n\n")

# Definir las métricas que deseas calcular
scoring = ['accuracy', 'precision', 'f1']

# Realizar la validación cruzada con 9 pliegues
cv = StratifiedKFold(n_splits=9, shuffle=True, random_state=42)

# Realizar la validación cruzada y obtener los resultados
results = cross_validate(naive_bayes, train_x, target_train, cv=cv, scoring=scoring)

# Acceder a los puntajes para cada pliegue y métrica
accuracy_scores = results['test_accuracy']
precision_scores = results['test_precision']
f1_scores = results['test_f1']

# Imprimir los puntajes para cada pliegue y métrica
for fold_idx, (accuracy, precision, f1) in enumerate(zip(accuracy_scores, precision_scores, f1_scores)):
    print(f"Pliegue {fold_idx+1}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"F1-score: {f1}")
    print("-----------------------------------")


Train accuracy:   97.21%


Pliegue 1:
Accuracy: 0.90234375
Precision: 0.8992248062015504
F1-score: 0.9027237354085603
-----------------------------------
Pliegue 2:
Accuracy: 0.8671875
Precision: 0.8175675675675675
F1-score: 0.8768115942028986
-----------------------------------
Pliegue 3:
Accuracy: 0.90625
Precision: 0.8939393939393939
F1-score: 0.9076923076923077
-----------------------------------
Pliegue 4:
Accuracy: 0.8862745098039215
Precision: 0.8721804511278195
F1-score: 0.888888888888889
-----------------------------------
Pliegue 5:
Accuracy: 0.8745098039215686
Precision: 0.8529411764705882
F1-score: 0.8787878787878787
-----------------------------------
Pliegue 6:
Accuracy: 0.8941176470588236
Precision: 0.8854961832061069
F1-score: 0.8957528957528957
-----------------------------------
Pliegue 7:
Accuracy: 0.8745098039215686
Precision: 0.841726618705036
F1-score: 0.8796992481203008
-----------------------------------
Pliegue 8:
Accuracy: 0.8627450980392157
Precision: 0.83823

##### Naive Bayes - GridSearchCV for f1_score

In [ ]:
from sklearn.model_selection import GridSearchCV

# Crear el clasificador Naive Bayes
naive_bayes = MultinomialNB()

# Definir los parámetros a explorar en la búsqueda en cuadrícula
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 5.0],
    'fit_prior': [True, False]
}

# Realizar la búsqueda en cuadrícula con validación cruzada
grid_search = GridSearchCV(estimator=naive_bayes, param_grid=param_grid, cv=9, scoring='f1')
grid_search.fit(train_x, target_train)

# Obtener los mejores parámetros y el mejor resultado
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Mejores parámetros encontrados:")
print(best_params)
print("\nMejor puntuación (accuracy):")
print(best_score)

# Obtener el mejor modelo entrenado con los mejores parámetros
best_model = grid_search.best_estimator_

# Predecir las etiquetas de los datos de entrenamiento
train_predictions = best_model.predict(train_x)
train_accuracy = accuracy_score(target_train, train_predictions)
print(f"\nTrain accuracy: {train_accuracy:0.2%}\n")

# Definir las métricas que deseas calcular
scoring = ['accuracy', 'precision', 'recall', 'f1']

# Realizar la validación cruzada con 9 pliegues
cv = StratifiedKFold(n_splits=9, shuffle=True, random_state=42)

# Realizar la validación cruzada y obtener los resultados con el mejor modelo
results = cross_validate(best_model, train_x, target_train, cv=cv, scoring=scoring)

# Acceder a los puntajes para cada pliegue y métrica
accuracy_scores = results['test_accuracy']
precision_scores = results['test_precision']
recall_scores = results['test_recall']
f1_scores = results['test_f1']

# Imprimir los puntajes para cada pliegue y métrica
for fold_idx, (accuracy, precision, recall, f1) in enumerate(zip(accuracy_scores, precision_scores, recall_scores, f1_scores)):
    print(f"Pliegue {fold_idx+1}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall-score: {recall}")
    print(f"F1-score: {f1}")
    print("-----------------------------------")


Mejores parámetros encontrados:
{'alpha': 0.1, 'fit_prior': True}

Mejor puntuación (accuracy):
0.8925644759614971

Train accuracy: 98.74%

Pliegue 1:
Accuracy: 0.90625
Precision: 0.9193548387096774
Recall-score: 0.890625
F1-score: 0.9047619047619047
-----------------------------------
Pliegue 2:
Accuracy: 0.890625
Precision: 0.852112676056338
Recall-score: 0.9453125
F1-score: 0.8962962962962963
-----------------------------------
Pliegue 3:
Accuracy: 0.91796875
Precision: 0.928
Recall-score: 0.90625
F1-score: 0.9169960474308302
-----------------------------------
Pliegue 4:
Accuracy: 0.8823529411764706
Precision: 0.8769230769230769
Recall-score: 0.890625
F1-score: 0.883720930232558
-----------------------------------
Pliegue 5:
Accuracy: 0.8862745098039215
Precision: 0.8837209302325582
Recall-score: 0.890625
F1-score: 0.8871595330739299
-----------------------------------
Pliegue 6:
Accuracy: 0.8941176470588236
Precision: 0.904
Recall-score: 0.8828125
F1-score: 0.8932806324110673
----

##### Naive Bayes - GridSearchCV for f1_score - BaggingClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import BaggingClassifier
from sklearn.naive_bayes import MultinomialNB

# Crear el clasificador base Naive Bayes
naive_bayes = MultinomialNB()

# Definir los parámetros a explorar en la búsqueda en cuadrícula
param_grid = {
    'alpha': [0.1, 0.5, 1.0, 5.0],
    'fit_prior': [True, False]
}

# Realizar la búsqueda en cuadrícula con validación cruzada para el clasificador base
grid_search = GridSearchCV(estimator=naive_bayes, param_grid=param_grid, cv=9, scoring='f1')
grid_search.fit(train_x, target_train)

# Obtener los mejores parámetros y el mejor resultado para el clasificador base
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Mejores parámetros encontrados para el clasificador base:")
print(best_params)
print("\nMejor puntuación (accuracy) para el clasificador base:")
print(best_score)

# Obtener el mejor modelo entrenado con los mejores parámetros para el clasificador base
best_model_base = grid_search.best_estimator_

# Crear el clasificador Bagging utilizando el clasificador base Naive Bayes
# Entrenar el clasificador Bagging con los datos de entrenamiento
bagging_classifier = BaggingClassifier(base_estimator=best_model_base, n_estimators=50).fit(train_x, target_train)

# Predecir las etiquetas de los datos de entrenamiento
train_predictions = bagging_classifier.predict(train_x)
train_accuracy = accuracy_score(target_train, train_predictions)
print(f"\nTrain accuracy: {train_accuracy}\n")

# Definir las métricas que deseas calcular
scoring = ['accuracy', 'precision', 'recall', 'f1']

# Realizar la validación cruzada con 9 pliegues
cv = StratifiedKFold(n_splits=9, shuffle=True, random_state=42)

# Realizar la validación cruzada con el clasificador Bagging y obtener los resultados
results = cross_validate(bagging_classifier, train_x, target_train, cv=cv, scoring=scoring)

# Acceder a los puntajes para cada pliegue y métrica
accuracy_scores = results['test_accuracy']
precision_scores = results['test_precision']
recall_scores = results['test_recall']
f1_scores = results['test_f1']

# Imprimir los puntajes para cada pliegue y métrica
for fold_idx, (accuracy, precision, recall, f1) in enumerate(zip(accuracy_scores, precision_scores, recall_scores, f1_scores)):
    print(f"Pliegue {fold_idx+1}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"Recall-score: {recall}")
    print(f"F1-score: {f1}")
    print("-----------------------------------")


Mejores parámetros encontrados para el clasificador base:
{'alpha': 0.1, 'fit_prior': True}

Mejor puntuación (accuracy) para el clasificador base:
0.8925644759614971


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(



Train accuracy: 0.9869451697127938



/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_base.py:166: FutureWarning: `base_estimator` was renamed to `estimator` in version 1.2 and will be removed in 1.4.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/en

Pliegue 1:
Accuracy: 0.91015625
Precision: 0.92
Recall-score: 0.8984375
F1-score: 0.9090909090909092
-----------------------------------
Pliegue 2:
Accuracy: 0.890625
Precision: 0.852112676056338
Recall-score: 0.9453125
F1-score: 0.8962962962962963
-----------------------------------
Pliegue 3:
Accuracy: 0.91796875
Precision: 0.9212598425196851
Recall-score: 0.9140625
F1-score: 0.9176470588235294
-----------------------------------
Pliegue 4:
Accuracy: 0.8784313725490196
Precision: 0.8702290076335878
Recall-score: 0.890625
F1-score: 0.8803088803088803
-----------------------------------
Pliegue 5:
Accuracy: 0.8980392156862745
Precision: 0.8984375
Recall-score: 0.8984375
F1-score: 0.8984375
-----------------------------------
Pliegue 6:
Accuracy: 0.8784313725490196
Precision: 0.8818897637795275
Recall-score: 0.875
F1-score: 0.8784313725490196
-----------------------------------
Pliegue 7:
Accuracy: 0.8901960784313725
Precision: 0.8721804511278195
Recall-score: 0.9133858267716536
F1-scor

In [ ]:
test_pred = bagging_classifier.predict(test_x)

test_accuracy = accuracy_score(target_test, test_pred)
test_precision = precision_score(target_test, test_pred)
test_recall = recall_score(target_test, test_pred)
test_f1 = f1_score(target_test, test_pred)
test_matrix = confusion_matrix(target_test, test_pred)

print(f"Test accuracy: {test_accuracy}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Test f1-score: {test_f1}")
print(f"Test Matrix: {test_matrix}")

Test accuracy: 0.890625
Test precision: 0.8968253968253969
Test recall: 0.8828125
Test f1-score: 0.8897637795275591
Test Matrix: [[115  13]
 [ 15 113]]


#### Support Vector Machine

In [ ]:
from sklearn.svm import SVC

In [ ]:
# Crear el clasificador SVM
svm_classifier = SVC(kernel="linear", C=50)

# Entrenar el clasificador con todos los datos de entrenamiento
svm_classifier.fit(train_x, target_train)

# Predecir las etiquetas de los datos de entrenamiento
train_predictions = svm_classifier.predict(train_x)
train_accuracy = accuracy_score(target_train, train_predictions)
print(f"Train accuracy:   {train_accuracy:0.2%}\n\n")

# Definir las métricas que deseas calcular
scoring = ['accuracy', 'precision', 'f1']

# Realizar la validación cruzada con 9 pliegues
cv = StratifiedKFold(n_splits=9, shuffle=True, random_state=42)

# Realizar la validación cruzada y obtener los resultados
results = cross_validate(svm_classifier, train_x, target_train, cv=cv, scoring=scoring)

# Acceder a los puntajes para cada pliegue y métrica
accuracy_scores = results['test_accuracy']
precision_scores = results['test_precision']
f1_scores = results['test_f1']

# Imprimir los puntajes para cada pliegue y métrica
for fold_idx, (accuracy, precision, f1) in enumerate(zip(accuracy_scores, precision_scores, f1_scores)):
    print(f"Pliegue {fold_idx+1}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"F1-score: {f1}")
    print("-----------------------------------")


Train accuracy:   100.00%


Pliegue 1:
Accuracy: 0.92578125
Precision: 0.9291338582677166
F1-score: 0.9254901960784313
-----------------------------------
Pliegue 2:
Accuracy: 0.8671875
Precision: 0.873015873015873
F1-score: 0.8661417322834646
-----------------------------------
Pliegue 3:
Accuracy: 0.89453125
Precision: 0.9105691056910569
F1-score: 0.8924302788844621
-----------------------------------
Pliegue 4:
Accuracy: 0.8666666666666667
Precision: 0.8916666666666667
F1-score: 0.8629032258064517
-----------------------------------
Pliegue 5:
Accuracy: 0.8313725490196079
Precision: 0.84
F1-score: 0.8300395256916997
-----------------------------------
Pliegue 6:
Accuracy: 0.8509803921568627
Precision: 0.8947368421052632
F1-score: 0.8429752066115702
-----------------------------------
Pliegue 7:
Accuracy: 0.8470588235294118
Precision: 0.8492063492063492
F1-score: 0.8458498023715415
-----------------------------------
Pliegue 8:
Accuracy: 0.8470588235294118
Precision: 0.84920634920634

In [ ]:
test_pred = svm_classifier.predict(test_x)

test_accuracy = accuracy_score(target_test, test_pred)
test_precision = precision_score(target_test, test_pred)
test_recall = recall_score(target_test, test_pred)
test_f1 = f1_score(target_test, test_pred)
test_matrix = confusion_matrix(target_test, test_pred)

print(f"Test accuracy: {test_accuracy:0.2%}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Test f1-score: {test_f1}")
print(f"Test Matrix: {test_matrix}")

Test accuracy: 87.50%
Test precision: 0.8870967741935484
Test recall: 0.859375
Test f1-score: 0.8730158730158729
Test Matrix: [[114  14]
 [ 18 110]]


#### Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
# Crear el clasificador de regresión logística
logistic_regression = LogisticRegression(max_iter=1000, class_weight="balanced")

# Entrenar el clasificador con todos los datos de entrenamiento
logistic_regression.fit(train_x, target_train)

# Predecir las etiquetas de los datos de entrenamiento
train_predictions = logistic_regression.predict(train_x)
train_accuracy = accuracy_score(target_train, train_predictions)
print(f"Train accuracy:   {train_accuracy:0.2%}\n\n")


# Definir las métricas que deseas calcular
scoring = ['accuracy', 'precision', 'f1']

# Realizar la validación cruzada con 9 pliegues
cv = StratifiedKFold(n_splits=9, shuffle=True, random_state=42)

# Realizar la validación cruzada y obtener los resultados
results = cross_validate(logistic_regression, train_x, target_train, cv=cv, scoring=scoring)

# Acceder a los puntajes para cada pliegue y métrica
accuracy_scores = results['test_accuracy']
precision_scores = results['test_precision']
f1_scores = results['test_f1']

# Imprimir los puntajes para cada pliegue y métrica
for fold_idx, (accuracy, precision, f1) in enumerate(zip(accuracy_scores, precision_scores, f1_scores)):
    print(f"Pliegue {fold_idx+1}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"F1-score: {f1}")
    print("-----------------------------------")

Train accuracy:   100.00%


Pliegue 1:
Accuracy: 0.92578125
Precision: 0.9504132231404959
F1-score: 0.9236947791164659
-----------------------------------
Pliegue 2:
Accuracy: 0.8828125
Precision: 0.8656716417910447
F1-score: 0.8854961832061069
-----------------------------------
Pliegue 3:
Accuracy: 0.89453125
Precision: 0.8914728682170543
F1-score: 0.8949416342412452
-----------------------------------
Pliegue 4:
Accuracy: 0.8823529411764706
Precision: 0.9152542372881356
F1-score: 0.8780487804878049
-----------------------------------
Pliegue 5:
Accuracy: 0.8941176470588236
Precision: 0.8740740740740741
F1-score: 0.897338403041825
-----------------------------------
Pliegue 6:
Accuracy: 0.8588235294117647
Precision: 0.896551724137931
F1-score: 0.8524590163934426
-----------------------------------
Pliegue 7:
Accuracy: 0.8549019607843137
Precision: 0.8515625
F1-score: 0.8549019607843137
-----------------------------------
Pliegue 8:
Accuracy: 0.8666666666666667
Precision: 0.8604651162

In [ ]:
test_pred = logistic_regression.predict(test_x)

test_accuracy = accuracy_score(target_test, test_pred)
test_precision = precision_score(target_test, test_pred)
test_recall = recall_score(target_test, test_pred)
test_f1 = f1_score(target_test, test_pred)
test_matrix = confusion_matrix(target_test, test_pred)

print(f"Test accuracy: {test_accuracy:0.2%}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Test f1-score: {test_f1}")
print(f"Test Matrix: {test_matrix}")

Test accuracy: 87.50%
Test precision: 0.8934426229508197
Test recall: 0.8515625
Test f1-score: 0.8719999999999999
Test Matrix: [[115  13]
 [ 19 109]]


In [ ]:
bagging_seed = bagging_classifier.random_state
print("Semilla utilizada por el BaggingClassifier:", bagging_seed)


Semilla utilizada por el BaggingClassifier: None


#### Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Crear el clasificador de Bosques Aleatorios
random_forest = RandomForestClassifier()

# Entrenar el clasificador con todos los datos de entrenamiento
random_forest.fit(train_x, target_train)

# Predecir las etiquetas de los datos de entrenamiento
train_predictions = random_forest.predict(train_x)
train_accuracy = accuracy_score(target_train, train_predictions)
print(f"Train accuracy:   {train_accuracy:0.2%}\n\n")

# Definir las métricas que deseas calcular
scoring = ['accuracy', 'precision', 'f1']

# Realizar la validación cruzada con 9 pliegues
cv = StratifiedKFold(n_splits=9, shuffle=True, random_state=42)

# Realizar la validación cruzada y obtener los resultados
results = cross_validate(random_forest, train_x, target_train, cv=cv, scoring=scoring)

# Acceder a los puntajes para cada pliegue y métrica
accuracy_scores = results['test_accuracy']
precision_scores = results['test_precision']
f1_scores = results['test_f1']

# Imprimir los puntajes para cada pliegue y métrica
for fold_idx, (accuracy, precision, f1) in enumerate(zip(accuracy_scores, precision_scores, f1_scores)):
    print(f"Pliegue {fold_idx+1}:")
    print(f"Accuracy: {accuracy}")
    print(f"Precision: {precision}")
    print(f"F1-score: {f1}")
    print("-----------------------------------")

Train accuracy:   100.00%


Pliegue 1:
Accuracy: 0.90234375
Precision: 0.9327731092436975
F1-score: 0.8987854251012145
-----------------------------------
Pliegue 2:
Accuracy: 0.84765625
Precision: 0.8503937007874016
F1-score: 0.8470588235294119
-----------------------------------
Pliegue 3:
Accuracy: 0.890625
Precision: 0.923728813559322
F1-score: 0.8861788617886179
-----------------------------------
Pliegue 4:
Accuracy: 0.8549019607843137
Precision: 0.8956521739130435
F1-score: 0.8477366255144032
-----------------------------------
Pliegue 5:
Accuracy: 0.8352941176470589
Precision: 0.8583333333333333
F1-score: 0.8306451612903225
-----------------------------------
Pliegue 6:
Accuracy: 0.8549019607843137
Precision: 0.9099099099099099
F1-score: 0.8451882845188285
-----------------------------------
Pliegue 7:
Accuracy: 0.8549019607843137
Precision: 0.8947368421052632
F1-score: 0.8464730290456431
-----------------------------------
Pliegue 8:
Accuracy: 0.8352941176470589
Precision: 0.8

In [ ]:
test_pred = random_forest.predict(test_x)

test_accuracy = accuracy_score(target_test, test_pred)
test_precision = precision_score(target_test, test_pred)
test_recall = recall_score(target_test, test_pred)
test_f1 = f1_score(target_test, test_pred)
test_matrix = confusion_matrix(target_test, test_pred)

print(f"Test accuracy: {test_accuracy:0.2%}")
print(f"Test precision: {test_precision}")
print(f"Test recall: {test_recall}")
print(f"Test f1-score: {test_f1}")
print(f"Test Matrix: {test_matrix}")

Test accuracy: 86.72%
Test precision: 0.8983050847457628
Test recall: 0.828125
Test f1-score: 0.8617886178861789
Test Matrix: [[116  12]
 [ 22 106]]
